In [23]:
import torch
import torch.nn as nn

import well_log_analysis.dataset
from torch.utils.data import DataLoader

import numpy as np
import sklearn
import sklearn.metrics

from sklearn.preprocessing import MinMaxScaler, RobustScaler

In [24]:
BATCH_SIZE = 32
SEQUENCE_LENGTH = 32
NUM_EPOCHS = 1

In [25]:
def train_test(train_wells, test_well):
    train_dataset = well_log_analysis.dataset.WellDataset(train_wells[0], 
                                    ['GR', 'DEN', 'RT', 'DTCO', 'DTSM'], 
                                    ['GR', 'DEN', 'RT', 'DTCO', 'DTSM'], 
                                    sequence_length=SEQUENCE_LENGTH)
    
    for i in range(1, len(train_wells)):
        train_dataset += well_log_analysis.dataset.WellDataset(train_wells[i], 
                                        ['GR', 'DEN', 'RT', 'DTCO', 'DTSM'], 
                                        ['GR', 'DEN', 'RT', 'DTCO', 'DTSM'], 
                                        sequence_length=SEQUENCE_LENGTH)
    test_dataset = well_log_analysis.dataset.WellDataset(test_well,
                                ['GR', 'DEN', 'RT', 'DTCO', 'DTSM'], 
                                ['GR', 'DEN', 'RT', 'DTCO', 'DTSM'], 
                                sequence_length=SEQUENCE_LENGTH)
    return train_dataset, test_dataset

In [26]:
class LSTMModel(nn.Module):
    def __init__(self, in_size, out_size, dropout=0.5, hidden_units=64, num_rnn_layers=4):
        super().__init__()
        self.num_layers = num_rnn_layers
        self.hidden_units = hidden_units
        self.fc1 = nn.Linear(3, 32)
        self.lstm = nn.LSTM(32, self.hidden_units, self.num_layers, dropout=dropout, batch_first=True)
        self.fc2 = nn.Linear(self.hidden_units, 2)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        batch_size = x.shape[0]
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_units).requires_grad_()
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_units).requires_grad_()
        x, (_, _) = self.lstm(x, (h0, c0))
        x = self.fc2(x)
        return x

In [33]:
def experiment(train_wells, test_well):
    train_dataset, test_dataset = train_test(train_wells, test_well)
    scaler = RobustScaler()
    
    train_dataset.copy_df = train_dataset.df.copy()
    train_dataset.df[train_dataset.return_sites] = scaler.fit_transform(train_dataset.df[train_dataset.return_sites])
    
    test_dataset.copy_df = train_dataset.df.copy()
    test_dataset.df[test_dataset.return_sites] = scaler.transform(test_dataset.df[train_dataset.return_sites])
    
    model = LSTMModel(3, 2)
    optim = torch.optim.Adam(model.parameters(), weight_decay=1e-4)
    
    train_loader = DataLoader(train_dataset, batch_size = BATCH_SIZE, drop_last=True, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size = 64)
    for epoch in range(NUM_EPOCHS):
        total_losses = []
        for batch in train_loader:
            optim.zero_grad()
            
            x, y = batch[:, :, :3].float(), batch[:, :, 3:].float()
            
            pred = model(x)
            
            loss = nn.SmoothL1Loss()(y, pred)
            total_losses.append(loss.detach().clone().cpu())
            
            loss.backward()
            optim.step()
            
        print("Epoch {}: {}".format(epoch, np.mean(total_losses)))
        
    model.eval()
    ground_truth = []
    preds = []
    
    for batch in test_loader:
        x, y = batch[:, :, :3].float(), batch[:, :, 3:].float()
        pred = model(x)
        ground_truth.append(y[:, -1, :])
        preds.append(pred[:, -1, :])
        
    ground_truth = torch.stack(ground_truth)[:, 0]
    preds = torch.stack(preds)[:, 0]
    print(ground_truth.shape, preds.shape)
    
    
        

In [35]:
dataset_paths = ['data/dataset1/J10025.las', 'data/dataset1/J10035.las', 'data/dataset1/J10038.las', 'data/dataset1/J10039.las', 'data/dataset1/J10051.las']


for path in dataset_paths:
    train_datasets = dataset_paths.copy()
    train_datasets.remove(path)
    experiment(train_datasets, path)

Epoch 0: 0.04780116304755211


RuntimeError: stack expects each tensor to be equal size, but got [64, 2] at entry 0 and [43, 2] at entry 66